# Entities as Experts

This notebook is a code implementation of the paper "Entities as Experts: Sparse Memory Access with Entity Supervision" by Févry, Baldini Soares, FitzGerald, Choi, Kwiatowski.

## Problem definition and high-level model description

We want to perform question answering on typical one-shot questions that require external knowledge or context. For example, in order to answer the question "Which country was Charles Darwin born in?" one needs some text providing answers on typical structured scenarios.

In this case, however, we want to rely on knowledge-graph extracted information. For example, in the question given here, we can prune out unrelated to the antropologist and evolution theorist Charles Darwins, e.g. Charles River, Darwin City etc. 

In the paper, the authors propose to augment BERT in the task of cloze-type question answering by leveraging an Entity Memory extracted from e.g. a Knoweldge Graph.

![Entity as Experts description](images/eae_highlevel.png)

The Entity Memory is a simple bunch of embeddings of entities extracted from a Knowledge Graph. Relationships are ignored (see the Facts as Experts paper and notebook to see how they could be used).

## Datasets

> We assume access to a corpus $D={(xi,mi)}$,where all entity mentions are detected but not necessarily  all  linked  to  entities.   We  use  English Wikipedia as our corpus, with a vocabulary of 1m entities. Entity links come from hyperlinks, leading to 32m 128 byte contexts containing 17m entity links.

In the appendix B, it is explained that:

> We build our training corpus of contexts paired with entity mention labels from the 2019-04-14 dump of English Wikipedia. We first divide each article into chunks of 500 bytes,resulting in a corpus of 32 million contexts withover 17 million entity mentions. We restrict our-selves  to  the  one  million  most  frequent  entities
(86% of the linked mentions).

Given that the dump 2019-04-14 is not available at the time of writing, we will adopt the revision 2020-11-01.

Entities are thus partially extracted by link annotations (e.g. they associate with each token a mention if that token belongs to a wikipedia url).

## Mention Detection

> In addition to the Wikipedia links, we annotaten each sentence with unlinked mention spans using the mention detector from Section 2.2

The mention detection head discussed in Section 2.2 is a simple BIO sequence: each token is annotated with a B (beginning), I (inside) or O (outside) if they are respectivelly beginning, inside or outside of a mention. The reason why we use both BIO and EL is to avoid inconsistencies.

There is a catch. In the paper, they explain they used Google NLP APIs to perform entity detection and linking on large-scale Wikipedia entries, that is, to have a properly annotated Wikipedia dataset. We are going to use simple Wikipedia hyperlinks insteal (TODO: consider adding spacy annotation?).

NOTE FOR MYSELF: We don't *actually* perform entity linking here - as in, we don't train a classifier from the first $l_0$ layers. Instead we'll build pseudo embeddings and try to find the entity that best matches a pseudo embedding (see later).

HOWEVER, we do need that when training. The whole idea is that we get supervised data when training, however it is not always the case that we have this data at hand (see: TriviaQA, WebQuestions, ...).

## Entity Memory

The idea is pretty simple: we have as input $X_1$ and the mention spans $m_i = (e_i, s_{m_i}, t_{m_i})$ . Those are given as input. We don't care about e_i for the embedding calculation, but we DO care for the loss definition.

(Glossing over the Entity Memory calculation...)

When the entity detection is supervised, our obtained entity should be close to the found pseudo entity embedding.

$$
ELLoss = \sum_{m_i} \alpha_i \cdot \mathbb{1}_{e_{m_i} \ne e_{\emptyset}} , \qquad \alpha = softmax(E \cdot h_{m_i})
$$

($E$ is our `EntEmbed`, so $E * h_{m_i}$ is a vector, so $\alpha$ is a vector too (of shape $N$), and $h_{m_i}$ is a "pseudo entity embedding" ).

## Chunking
- In theory we should split articles by chunks of 500 bytes (assuming unicode encoding), and contexts are only 128 tokens long. For simplicity by now we only limit ourselves to the first paragraph only.

## Tokenization:

- BERT Tokenizer (e.g. Wordpiece) using lowercase vocabulary, limited to 128 distinct word-piece tokens.

## Learning hyperparameters

For pretraining:

> We use ADAM with a learning rate of 1e-4.  We apply warmup for the first 5% of training, decaying the learning rate afterwards.  We also apply gradient clipping with a norm of 1.0

Since the decaying rate is not provided, we test with 3e-5 which seems quite standard.

## Evaluation

To evaluate:

- SQuAD
- TriviaQA
- MetaQA
- (Colla?)

#### Wikipedia

In [ ]:
from tools.dataloaders import WikipediaCBOR
from models import EntityAsExperts, EaEForQuestionAnswering

In [2]:
wikipedia_cbor = WikipediaCBOR("wikipedia/car-wiki2020-01-01/enwiki2020.cbor", "wikipedia/car-wiki2020-01-01/partitions",
                                       # top 2% most frequent items,  roughly at least 100 occurrences, with a total of  ~ 20000 entities
                                       #cutoff_frequency=0.02, recount=True 
                                    # TODO: is this representative enough?
)

Loaded from cache


In [3]:
FULL_FINETUNING=False
BATCH_SIZE=8
NUM_WORKERS=8

In [4]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler

# https://discuss.pytorch.org/t/random-subset-from-dataloader-unique-random-numbers/43430/6
import numpy as np
np.random.seed(42)

if not FULL_FINETUNING:
    wiki_dev_size = int(0.1*len(wikipedia_cbor))

else:
    wiki_dev_size = len(wikipedia_cbor)
    
wiki_dev_indices = np.random.choice(len(wikipedia_cbor), size=wiki_dev_size)

# 80/20% split
wiki_train_size = int(0.8*wiki_dev_size)
wiki_validation_size = wiki_dev_size - wiki_train_size

wiki_train_indices, wiki_validation_indices = wiki_dev_indices[:wiki_train_size], wiki_dev_indices[wiki_train_size:]
wiki_train_sampler = SubsetRandomSampler(wiki_train_indices, generator=torch.Generator().manual_seed(42))
wiki_validation_sampler = SubsetRandomSampler(wiki_validation_indices, generator=torch.Generator().manual_seed(42))

wiki_train_dataloader = DataLoader(wikipedia_cbor, sampler=wiki_train_sampler, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
wiki_validation_dataloader = DataLoader(wikipedia_cbor, sampler=wiki_validation_sampler, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
# wikipedia_cbor_train, wikipedia_cbor_validation = random_split(wikipedia_


## Model

### Load and finetune the model

In [8]:
from transformers import BertForMaskedLM, BertForTokenClassification
from models import EntityAsExperts, EaEForQuestionAnswering
from models.training import load_model, save_models, train_model, get_optimizer, get_schedule
model_masked_lm = BertForMaskedLM.from_pretrained('bert-base-uncased')

In [6]:
embedding_size = 768 # BERT-base
entity_embedding_size = 256
l0 = 4
l1 = 8
pretraining_model = EntityAsExperts(model_masked_lm, l0, l1,  wikipedia_cbor.max_entity_num,
                                     entity_embedding_size)

In [7]:
pretraining_epochs = 1
pretraining_optimizer = get_optimizer(pretraining_model)
pretraining_schedule = get_schedule(pretraining_epochs, pretraining_optimizer, wiki_train_dataloader)

# TODO: automatically send the model to the device AND provide on/off switches for it

def wiki_load_batch(batch):
    # Nothing interesting...
    # TODO: do we still need this?
    return batch

train_model(pretraining_model.to('cuda'), wiki_train_dataloader,
                wiki_validation_dataloader, lambda x: x, pretraining_optimizer,
                pretraining_schedule, pretraining_epochs, None)

  0%|          | 0/722 [00:00<?, ?it/s]

Average train loss at epoch 0: 0.27000342697096047


100%|██████████| 722/722 [01:52<00:00,  6.44it/s]

Average eval loss at epoch 0: 0.10770124641254833


(0.27000342697096047, 0.10770124641254833)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from typing import List

def plot_losses(train_loss: List[int], validation_loss: List[int], title="", show_minimum=False):
    
    epochs = len(train_loss)
    
    fig = figure(num=None, figsize=(8, 6), dpi=120, facecolor='w', edgecolor='k')
    
    plt.xticks(np.arange(epochs))
    
    plt.plot(train_loss, '-')
    plt.plot(validation_loss, '--')
    
    min_point = np.argmin(validation_loss)
    
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    
    plt.legend(["Train loss", "Validation loss"])
    
    if show_minimum:
        plt.scatter([min_point], validation_loss[min_point], c='r', marker='o', )
        plt.figtext(.5, .0, "Red dot represents the minimum validation loss value reached", ha='center')
    plt.figure(figsize=(16, 10))
    plt.show()

    
plot_losses(train_losses, validation_losses, "Pretraining stage loss", True)

In [ ]:
pretraining_model = load_model(Pretraining, "pretrained_eae_minimal", model_masked_lm, l0, l1,
                               wikipedia_cbor.max_entity_num, entity_embedding_size)

In [10]:
squad_qa_model = EaEForQuestionAnswering(pretraining_model)

save_models(squad_qa_untrained=squad_qa_model)

In [11]:
from tools.dataloaders import SQuADDataloader

squad_dataset = SQuADDataloader()
squad_train_dataset = squad_dataset.train_dataset
squad_train_dataloader = DataLoader(squad_train_dataset, batch_size=8)
squad_validation_dataset = squad_dataset.validation_dataset
squad_validation_dataloader = DataLoader(squad_validation_dataset, batch_size=8)

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7)


In [20]:
squad_epochs = 1
squad_optimizer = get_optimizer(pretraining_model)
squad_schedule = get_schedule(pretraining_epochs, pretraining_optimizer, wiki_train_dataloader)

# TODO: automatically send the model to the device AND provide on/off switches for it

def wiki_load_batch(batch):
    return tuple(batch[x] for x in ["input_ids", "answer_start", "answer_end",
                                        "attention_mask", "token_type_ids"])

train_model(squad_qa_model.to('cuda'), squad_train_dataloader,
                squad_validation_dataloader, pretraining_optimizer,
                pretraining_schedule, pretraining_epochs, None)

In [20]:
train_losses = []
validation_losses = []

squad_epochs = 5

squad_optimizer = get_optimizer(squad_qa_model)
squad_schedule = get_schedule(squad_epochs, squad_optimizer, squad_train_dataloader)

for epoch in range(squad_epochs):
    squad_qa_model.train()
    
    total_loss = 0
    
    for batch in tqdm(squad_train_dataloader):
        b_input_ids, b_answer_start, b_answer_end, b_attns, b_token_type_ids = \
            tuple(batch[x].to('gpu') for x in ["input_ids", "answer_start", "answer_end",
                                                "attention_mask", "token_type_ids"])
        squad_qa_model.zero_grad()
        loss, start_logits, end_logits = squad_qa_model(b_input_ids, b_attns, b_token_type_ids,
                                           b_answer_start, b_answer_end)
        loss.backward()
        total_loss += loss.item()
        
        clip_grad_norm_(parameters=squad_qa_model.parameters(),
                            max_norm=max_grad_norm)
    
        squad_optimizer.step()
        squad_schedule.step()
        
    avg_train_loss = total_loss / len(squad_train_dataloader)
    tqdm.write(f"Average train loss at epoch {epoch}: {avg_train_loss}")
    
    train_losses.append(avg_train_loss)
    
    squad_qa_model.eval()
    
    total_loss = 0
    
    for batch in tqdm(squad_validation_dataloader):
        
        b_input_ids, b_answer_start, b_answer_end, b_attns, b_token_type_ids = \
            tuple(batch[x].to(device) for x in ["input_ids", "answer_start", "answer_end",
                                                "attention_mask", "token_type_ids"])
        
        with torch.no_grad():
            loss, start_logits, end_logits = squad_qa_model(b_input_ids, b_attns, b_token_type_ids,
                                           b_answer_start, b_answer_end)        
            total_loss += loss.item()
        
    
    avg_validation_loss = total_loss / len(squad_validation_dataloader)
    validation_losses.append(avg_validation_loss)
    tqdm.write(f"Average eval loss at epoch {epoch}: {avg_validation_loss}")

  3%|▎         | 306/10950 [01:16<44:29,  3.99it/s]


RuntimeError: stack expects each tensor to be equal size, but got [518] at entry 0 and [520] at entry 2

In [22]:
print(type(squad_optimizer))



<class 'transformers.optimization.AdamW'>


NameError: name 'adamW' is not defined

In [ ]:
def train_model(
    model: Module,
    train_dataloader: DataLoader,
    validation_dataloader: DataLoader,
    load_from_dataloader,,
    epochs):
    """
    Train a model
    """
    
    train_losses = []
    validation_losses = []

    squad_epochs = 5

    squad_optimizer = get_optimizer(squad_qa_model)
    squad_schedule = get_schedule(squad_epochs, squad_optimizer, squad_train_dataloader)

    for epoch in range(squad_epochs):
        squad_qa_model.train()

        total_loss = 0

        for batch in tqdm(squad_train_dataloader):
            b_input_ids, b_answer_start, b_answer_end, b_attns, b_token_type_ids = \
                tuple(batch[x].to(device) for x in ["input_ids", "answer_start", "answer_end",
                                                    "attention_mask", "token_type_ids"])
            squad_qa_model.zero_grad()
            loss, start_logits, end_logits = squad_qa_model(b_input_ids, b_attns, b_token_type_ids,
                                               b_answer_start, b_answer_end)
            loss.backward()
            total_loss += loss.item()

            clip_grad_norm_(parameters=squad_qa_model.parameters(),
                                max_norm=max_grad_norm)

            squad_optimizer.step()
            squad_schedule.step()

        avg_train_loss = total_loss / len(squad_train_dataloader)
        tqdm.write(f"Average train loss at epoch {epoch}: {avg_train_loss}")

        train_losses.append(avg_train_loss)

        squad_qa_model.eval()

        total_loss = 0

        for batch in tqdm(squad_validation_dataloader):

            b_input_ids, b_answer_start, b_answer_end, b_attns, b_token_type_ids = \
                tuple(batch[x].to(device) for x in ["input_ids", "answer_start", "answer_end",
                                                    "attention_mask", "token_type_ids"])

            with torch.no_grad():
                loss, start_logits, end_logits = squad_qa_model(b_input_ids, b_attns, b_token_type_ids,
                                               b_answer_start, b_answer_end)        
                total_loss += loss.item()


        avg_validation_loss = total_loss / len(squad_validation_dataloader)
        validation_losses.append(avg_validation_loss)
        tqdm.write(f"Average eval loss at epoch {epoch}: {avg_validation_loss}")